In [1]:
# Import the modules
import datetime

import numpy as np
import spiceypy

In [2]:
# Load the SPICE kernels via a meta file
spiceypy.furnsh('kernel_meta.txt')

# Create an initial date-time object that is converted to a string
datetime_utc = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

# Convert to Ephemeris Time (ET) using the SPICE function utc2et
datetime_et = spiceypy.utc2et(datetime_utc)

In [3]:
# ECLIPJ2000_DE405 and ECLIPJ2000 appear to be similar?! A transformation
# matrix between both coordinate systems (for state vectors) should be
# consequently the identity matrix
mat = spiceypy.sxform(instring='ECLIPJ2000_DE405', \
                      tostring='ECLIPJ2000', \
                      et=datetime_et)

# Let's print the transformation matrix row-wise (spoiler alert: it is the
# identity matrix)
print('Transformation matrix between ECLIPJ2000_DE405 and ECLIPJ2000')
for mat_row in mat:
    print(f'{np.round(mat_row, 2)}')

Transformation matrix between ECLIPJ2000_DE405 and ECLIPJ2000
[1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0.]
[ 0. -0.  1.  0.  0.  0.]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 1. 0.]
[ 0.  0.  0.  0. -0.  1.]


In [4]:
# Compute the state vector of Ceres in ECLIPJ2000 as seen from the Sun
ceres_state_vector, _ = spiceypy.spkgeo(targ=2000001, \
                                        et=datetime_et, \
                                        ref='ECLIPJ2000',
                                        obs=10)

In [5]:
# Get the G*M value for the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

gm_sun = gm_sun_pre[0]

In [6]:
# Compute the orbital elements of Ceres using the computed state vector
ceres_orbital_elements = spiceypy.oscltx(state=ceres_state_vector, \
                                         et=datetime_et, \
                                         mu=gm_sun)

# Set and convert the semi-major axis and perihelion from km to AU
ceres_semi_major_au = spiceypy.convrt(ceres_orbital_elements[9], \
                                      inunit='km', outunit='AU')
ceres_perihelion_au = spiceypy.convrt(ceres_orbital_elements[0], \
                                      inunit='km', outunit='AU')

# Set the eccentricity
ceres_ecc = ceres_orbital_elements[1]

# Set and convert miscellaneous angular values from radians to degrees:
# inc: Inclination
# lnode: Longitude of ascending node
# argp: Argument of perihelion
ceres_inc_deg = np.degrees(ceres_orbital_elements[2])
ceres_lnode_deg = np.degrees(ceres_orbital_elements[3])
ceres_argp_deg = np.degrees(ceres_orbital_elements[4])

# Set the orbit period. Convert from seconds to years
ceres_orb_time_years = ceres_orbital_elements[10] / (86400.0 * 365.0)

In [7]:
# Compare the results with the data from the Minor Planet Center
# https://www.minorplanetcenter.net/dwarf_planets

# Print the results next to the MPC results
print('Ceres\' Orbital Elements')
print(f'Semi-major axis in AU: {round(ceres_semi_major_au, 2)} (MPC: 2.77)')
print(f'Perihelion in AU: {round(ceres_perihelion_au, 2)} (MPC: 2.55)')

print(f'Eccentricity: {round(ceres_ecc, 2)} (MPC: 0.08)')

print(f'Inclination in degrees: {round(ceres_inc_deg, 1)} (MPC: 10.6)')
print(f'Long. of. asc. node in degrees: {round(ceres_lnode_deg, 1)} ' \
      '(MPC: 80.3)')
print(f'Argument of perih. in degrees: {round(ceres_argp_deg, 1)} ' \
      '(MPC: 73.7)')

print(f'Orbit period in years: {round(ceres_orb_time_years, 2)} ' \
      '(MPC: 4.60)')

Ceres' Orbital Elements
Semi-major axis in AU: 2.77 (MPC: 2.77)
Perihelion in AU: 2.55 (MPC: 2.56)
Eccentricity: 0.08 (MPC: 0.08)
Inclination in degrees: 10.6 (MPC: 10.6)
Long. of. asc. node in degrees: 80.3 (MPC: 80.3)
Argument of perih. in degrees: 73.7 (MPC: 73.6)
Orbit period in years: 4.6 (MPC: 4.61)


In [8]:
# Convert the orbital elements back to the state vector
ceres_state_re = spiceypy.conics([ceres_orbital_elements[0], \
                                  ceres_orbital_elements[1], \
                                  ceres_orbital_elements[2], \
                                  ceres_orbital_elements[3], \
                                  ceres_orbital_elements[4], \
                                  ceres_orbital_elements[5], \
                                  ceres_orbital_elements[6], \
                                  gm_sun], datetime_et)

print('State vector of Ceres from the kernel:\n' \
      f'{ceres_state_vector}')
print('State vector of Ceres based on the determined orbital elements:\n' \
      f'{ceres_state_re}')
print('\n')

State vector of Ceres from the kernel:
[ 1.92000346e+08  3.64164086e+08 -2.38671514e+07 -1.61746303e+01
  7.14299335e+00  3.20566283e+00]
State vector of Ceres based on the determined orbital elements:
[ 1.92000346e+08  3.64164086e+08 -2.38671514e+07 -1.61746303e+01
  7.14299335e+00  3.20566283e+00]


